In [1]:
# %load_ext autoreload
# %autoreload 2

import os
import sys
os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [2]:
boiling_plan_df = pd.read_csv('2020.12.13 boiling_plan.csv')
boiling_plan_df['sku'] = boiling_plan_df['sku'].apply(cast_sku)
# boiling_plan_df['boiling'] = boiling_plan_df['boiling'].apply(cast_boiling)
boiling_plan_df['boiling'] = boiling_plan_df['sku'].apply(lambda sku: sku.boilings[0])
boiling_plan_df.sort_values(by='id')

def add_push_with_props(parent, block):
    return add_push(parent, block, new_props=block.props.get_all_props())


boiling_grp = boiling_plan_df[boiling_plan_df['id'] == 3]
boiling_grp['boiling_form_factor'] = boiling_grp['sku'].apply(lambda sku:sku.boiling_form_factors[0])

# mark groups
cur_group = 0
cur_form_factor = None
group_values = []
for i, row in boiling_grp.iterrows():
    if row['boiling_form_factor'] != cur_form_factor:
        cur_group += 1
        cur_form_factor = row['boiling_form_factor']
    group_values.append(cur_group)
boiling_grp['group'] = group_values

boiling_model = boiling_grp.iloc[0]['boiling']

<ipython-input-2-80d3cf80b653>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_grp['boiling_form_factor'] = boiling_grp['sku'].apply(lambda sku:sku.boiling_form_factors[0])
<ipython-input-2-80d3cf80b653>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_grp['group'] = group_values


In [3]:
maker = BlockMaker(default_push_func=dummy_push)
make = maker.make

with make('melting'):
    with make('servings', h=1, push_func=dummy_push_y):
        make('serving', time_size=boiling_model.meltings.serving_time)
    
    labels = make('labels', h=1, push_func=dummy_push_y).block
    meltings = make('meltings', h=1, push_func=dummy_push_y).block
    coolings = make('coolings', h=1, push_func=dummy_push_y).block
        
    for group, grp in boiling_grp.groupby('group'):
        if group > 1:
            # non-first group - reconfigure time
            dummy_push(meltings, Block('melting_configuration', size=1))
        
        melting_process = dummy_push(meltings, Block('melting_process', time_size=custom_round(grp['kg'].sum() / boiling_model.meltings.speed * 60, 5, 'ceil')))
        
        # add cooling
        cooling_process = Block('cooling_process', t=melting_process.beg())
        cooling_maker = BlockMaker(cooling_process, default_push_func=dummy_push)
        cooling_make = cooling_maker.make
        
        with cooling_make('start'):
            if boiling_model.boiling_type == 'water':
                cooling_make('cooling', time_size=boiling_model.meltings.first_cooling_time)
                cooling_make('cooling', time_size=boiling_model.meltings.second_cooling_time)
            elif boiling_model.boiling_type == 'salt':
                cooling_make('cooling', time_size=boiling_model.meltings.salting_time)
        
        with cooling_make('finish'):
            cooling_make('cooling', time_size=melting_process.props['time_size'])
        
        add_push(coolings, cooling_process)

In [4]:
maker.root

root (0, 4] x (0, 30]
  melting (0, 4] x (0, 30]
    servings (0, 1] x (0, 6]
      serving (0, 1] x (0, 6]
    labels (1, 2] x (0, 0]
    meltings (2, 3] x (0, 15]
      melting_process (2, 3] x (0, 7]
      melting_configuration (2, 3] x (7, 8]
      melting_process (2, 3] x (8, 15]
    coolings (3, 4] x (0, 30]
      cooling_process (3, 4] x (0, 22]
        start (3, 4] x (0, 15]
          cooling (3, 4] x (0, 5]
          cooling (3, 4] x (5, 15]
        finish (3, 4] x (15, 22]
          cooling (3, 4] x (15, 22]
      cooling_process (3, 4] x (8, 30]
        start (3, 4] x (8, 23]
          cooling (3, 4] x (8, 13]
          cooling (3, 4] x (13, 23]
        finish (3, 4] x (23, 30]
          cooling (3, 4] x (23, 30]